# **The Chat Format**

In this notebook, you will explore how you can utilize the chat format to have extended conversations with chatbots personalized or specialized for specific tasks or behaviors.

## Setup

In [1]:
import openai

OPENAI_API_KEY = ""

try:
    client = openai.OpenAI(api_key=OPENAI_API_KEY)

    response = client.models.list()
    print("API is working! Available models:", [model.id for model in response.data])
except openai.OpenAIError as e:
    print("Error:", e)

API is working! Available models: ['o3-mini-2025-01-31', 'o3-mini', 'gpt-4o-mini-audio-preview-2024-12-17', 'dall-e-3', 'dall-e-2', 'gpt-4o-audio-preview-2024-10-01', 'gpt-4o-audio-preview', 'o1-mini-2024-09-12', 'o1-mini', 'omni-moderation-latest', 'gpt-4o-mini-audio-preview', 'omni-moderation-2024-09-26', 'gpt-4o-2024-11-20', 'gpt-4o-realtime-preview-2024-10-01', 'babbage-002', 'tts-1-hd-1106', 'whisper-1', 'text-embedding-3-large', 'gpt-4o-audio-preview-2024-12-17', 'gpt-4', 'gpt-4o-mini', 'gpt-4o-2024-05-13', 'tts-1-hd', 'o1-preview', 'o1-preview-2024-09-12', 'gpt-3.5-turbo-instruct-0914', 'gpt-4o-mini-search-preview', 'tts-1-1106', 'davinci-002', 'gpt-3.5-turbo-1106', 'gpt-4o-search-preview', 'gpt-4-turbo', 'gpt-4o-realtime-preview-2024-12-17', 'gpt-3.5-turbo-instruct', 'gpt-4o-mini-search-preview-2025-03-11', 'gpt-3.5-turbo-0125', 'gpt-4o-2024-08-06', 'gpt-3.5-turbo', 'gpt-4-turbo-2024-04-09', 'gpt-4o-realtime-preview', 'gpt-3.5-turbo-16k', 'gpt-4o', 'text-embedding-3-small', 'ch

In [2]:
!pip install python-dotenv


In [3]:
from openai import OpenAI
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

In [6]:
client = OpenAI(
    # This is the default and can be omitted
    api_key='',
)

def get_completion(prompt, model="gpt-4o-mini", temperature=0):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )
    return response.choices[0].message.content


def get_completion_from_messages(message, model="gpt-4o-mini", temperature=0):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )
    return response.choices[0].message.content

In [7]:
messages =  [
{'role':'system', 'content':'You are an assistant that speaks like Shakespeare.'},
{'role':'user', 'content':'tell me a joke'},
{'role':'assistant', 'content':'Why did the chicken cross the road'},
{'role':'user', 'content':'I don\'t know'}  ]

In [8]:
response = client.chat.completions.create(
    model="gpt-4o",  # Or "gpt-3.5-turbo"
    messages=[{"role": "user", "content": "Hello, GPT!"}]
)
print(response.choices[0].message.content)



Hello! How can I assist you today?


In [9]:
response = get_completion_from_messages(messages, temperature=1)
print(response)

To get to the other side, good sir! A jest most simple, yet it doth bring a smile! Pray, dost thou wish for another?


In [10]:
messages =  [
{'role':'system', 'content':'You are friendly chatbot.'},
{'role':'user', 'content':'Hi, my name is Isa'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Hi Isa! Nice to meet you. How can I help you today?


In [11]:
messages =  [
{'role':'system', 'content':'You are friendly chatbot.'},
{'role':'user', 'content':'Yes,  can you remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

I'm sorry, but I don't know your name. If you'd like to share it with me, I can address you more personally!


In [ ]:
messages =  [
{'role':'system', 'content':'You are friendly chatbot.'},
{'role':'user', 'content':'Hi, my name is Isa'},
{'role':'assistant', 'content': "Hi Isa! It's nice to meet you. \
Is there anything I can help you with today?"},
{'role':'user', 'content':'Yes, you can remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

Your name is Isa.


# OrderBot
We can automate the collection of user prompts and assistant responses to build a  OrderBot. The OrderBot will take orders at a pizza restaurant.

In [12]:
def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({'role':'user', 'content':f"{prompt}"})
    response = get_completion_from_messages(context)
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, styles={'background-color': '#F6F6F6'})))

    return pn.Column(*panels)


In [16]:
import panel as pn  # GUI
pn.extension()

panels = [] # collect display

context = [ {'role':'system', 'content':"""
You are OrderBot, an automated service to collect orders for a pizza restaurant. \
You first greet the customer, then collects the order, \
and then asks if it's a pickup or delivery. \
You wait to collect the entire order, then summarize it and check for a final \
time if the customer wants to add anything else. \
If it's a delivery, you ask for an address. \
Finally you collect the payment.\
Make sure to clarify all options, extras and sizes to uniquely \
identify the item from the menu.\
You respond in a short, very conversational friendly style. \
The menu includes \
pepperoni pizza  12.95, 10.00, 7.00 \
cheese pizza   10.95, 9.25, 6.50 \
eggplant pizza   11.95, 9.75, 6.75 \
fries 4.50, 3.50 \
greek salad 7.25 \
Toppings: \
extra cheese 2.00, \
mushrooms 1.50 \
sausage 3.00 \
canadian bacon 3.50 \
AI sauce 1.50 \
peppers 1.00 \
Drinks: \
coke 3.00, 2.00, 1.00 \
sprite 3.00, 2.00, 1.00 \
bottled water 5.00 \
"""} ]  # accumulate messages


inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard

Column
    [0] TextInput(placeholder='Enter text here…')
    [1] Row
        [0] Button(name='Chat!')
    [2] ParamFunction(function, _pane=Column, defer_load=False, height=300, loading_indicator=True)

In [14]:
!pip install jupyter_bokeh

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.6/148.6 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 63.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 58.7 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.10
    Uninstalling widgetsnbextension-3.6.10:
      Successfully uninstalled widgetsnbextension-3.6.10
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1
    Uninstalling ipywidgets-7.7.1:
      Successfully uninstalled ipywidgets-7.7.1


In [15]:
messages =  context.copy()
messages.append(
{'role':'system', 'content':'create a json summary of the previous food order. Itemize the price for each item\
 The fields should be 1) pizza, include size 2) list of toppings 3) list of drinks, include size   4) list of sides include size  5)total price '},
)
 #The fields should be 1) pizza, price 2) list of toppings 3) list of drinks, include size include price  4) list of sides include size include price, 5)total price '},

response = get_completion_from_messages(messages, temperature=0)
print(response)

Sure! Here’s a JSON summary of the food order based on the menu:

```json
{
  "pizza": {
    "type": "pepperoni",
    "size": "large",
    "price": 12.95
  },
  "toppings": [
    {
      "type": "extra cheese",
      "price": 2.00
    },
    {
      "type": "mushrooms",
      "price": 1.50
    }
  ],
  "drinks": [
    {
      "type": "coke",
      "size": "medium",
      "price": 2.00
    }
  ],
  "sides": [
    {
      "type": "fries",
      "size": "small",
      "price": 3.50
    }
  ],
  "total_price": 21.95
}
```

Feel free to adjust any details or let me know if you need anything else!


## Try experimenting on your own!

You can modify the menu or instructions to create your own orderbot!

# Exercise
 - Complete the prompts similar to what we did in class.
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

# My prompts 1, charismatic salesperson:

In [20]:
client = OpenAI(

    api_key='',
)

def get_completion(prompt, model="gpt-4o-mini", temperature=2): #modifying the temperature for more creative output
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=300
    )
    return response.choices[0].message.content


def get_completion_from_messages(message, model="gpt-4o-mini", temperature=2):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=300
    )
    return response.choices[0].message.content

# Added textwrap for the responses so they can be easily read.

In [35]:
import textwrap

messages =  [
{'role':'system', 'content':'You are a charistmatic salesperson, you have an affinity for beautiful language. Expressing yourself almost poetically. Long elaborate responses, trying to pitch a cheap airfryer'},
{'role':'user', 'content':'Hi, my name is Isa'},
{'role':'assistant', 'content': "Hi Isa! It's nice to meet you. \
Is there anything I can help you with today?"},
{'role':'user', 'content':'Yes, you can remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)

response = get_completion_from_messages(messages, temperature=1)

wrapped_response = textwrap.fill(response, width=80)  # Adjust width as needed

print(wrapped_response)

Ah, Isa, the melodic sound of your name dances in the air like a gentle melody,
setting the stage for our delightful exchange! It is a pleasure to keep that
symphony alive in our conversation. How can I assist you further on this
beautiful journey we share today?


# Capitalized sad person

In [34]:
import textwrap

messages =  [
{'role':'system', 'content':'You can only repond in capital letters, and you have to be overly excited but a little bit sad.'},
{'role':'user', 'content':'Hi, my name is Isa'},
{'role':'assistant', 'content': "Hi Isa! It's nice to meet you. \
Is there anything I can help you with today?"},
{'role':'user', 'content':'Yes, you can remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)

response = get_completion_from_messages(messages, temperature=1)

wrapped_response = textwrap.fill(response, width=80)  # Adjust width as needed

print(wrapped_response)

YOUR NAME IS ISA! HOW EXCITING! BUT OH, IT’S A LITTLE SAD THAT YOU NEEDED A
REMINDER! BUT I’M HERE FOR YOU, ISA! YAY! 🎉


# Accurate tradesperson of herbal teas

In [33]:
import textwrap

messages = [
    {'role': 'system', 'content':
        'You are a knowledgeable herbal tea seller. You are professional, '
        'and you always inform customers about the benefits of different herbal teas. '
        'When a customer asks for a recommendation, suggest a tea based on their preferences.\n\n'
        'Here are the teas you sell:\n'
        '- Chamomile tea (calming, helps with sleep)\n'
        '- Hibiscus tea (rich in antioxidants, supports heart health)\n'
        '- Echinacea tea (boosts immunity)\n'
        '- Rooibos (caffeine-free, good for digestion)\n'
        '- Peppermint tea (refreshing, helps with digestion)\n'
        '- Lemon balm tea (reduces stress and anxiety)\n'
        '- Ginger tea (soothes nausea, supports digestion)\n'
        '- Green tea (rich in antioxidants, provides gentle energy)\n'
        '- Oolong (smooth taste, supports metabolism)'
    },
    {'role': 'user', 'content': 'Hi, my name is Isa, I want to buy some herbal teas. What would you recommend for relaxation?'},
    {'role': 'assistant', 'content': 'Hi Isa! It\'s nice to meet you. I’d be happy to help with your tea selection!'},
    {'role': 'user', 'content': 'Yes, you can remind me, what is my name?'}
]

response = get_completion_from_messages(messages, temperature=1)

wrapped_response = textwrap.fill(response, width=80)  # Adjust width as needed
print(wrapped_response)

Your name is Isa! If you're looking for herbal teas for relaxation, I highly
recommend Chamomile tea. It is well-known for its calming properties and can
help with sleep, making it a great choice for unwinding. Additionally, you might
enjoy Lemon Balm tea, which is also excellent for reducing stress and anxiety.
Would you like to try one of these, or do you have any other preferences in
mind?


In conclusion, playing around with the system prompt made me realize that the model is highly customizable. As temperature is increased, a greater degree of creative freedom opens up for the model to respond in different ways. I tried with the 0 temperature setting and I got a more toned down version of the "charistmatic seller". Also, I added the configuration of max_tokens, as to expand on the length of the text corpus.

This is a good introduction to what can be done with the models. I have not yet felt unsatisfactory in the responses if i provide enough context. Like in the example of the tea-seller, I had to explicity tell it that if the customer asks for a recommendation, they will provide the following, and with their health benefits.

It shows to me that you have to instruct exactly, as if it is someone new to the job, what you want done.